# Аль-Кади Бакил Али Мохаммед

## Task-3: Russian sentiment classification

## Goal: Fine-tune a pre-trained Russian language model to classify comments as toxic or not toxic.

## Model: blanchefort/rubert-base-cased-sentiment

### Dataset: https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load dataset
df = pd.read_csv('./labeled.csv')
df.head()


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [3]:
df.shape

(14412, 2)

## class balance

In [4]:
print(df['toxic'].value_counts())

toxic
0.0    9586
1.0    4826
Name: count, dtype: int64


# Preprocess the dataset

In [5]:
df = df[['comment', 'toxic']].dropna()
df = df.rename(columns={'comment': 'text', 'toxic': 'label'})

# Convert label to int (in case it's float)
df['label'] = df['label'].astype(int)

df.head()

,text,label
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


# Train/test split

In [6]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

/media/bakil/5E509CE3509CC367/Study/Анализ ответов в свободной форме на вопросы социологических исследований/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Load tokenizer and model

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

In [9]:
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


/media/bakil/5E509CE3509CC367/Study/Анализ ответов в свободной форме на вопросы социологических исследований/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Tokenize

In [10]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)



ap: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2883/2883 [00:00<00:00, 18381.28 examples/s]

# Set format for PyTorch

In [11]:
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Training arguments

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    do_eval=True,  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=20,
    save_steps=500,
    eval_steps=500,
)

# Define accuracy metric 

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Trainer

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Step,Training Loss
20,1.323200
40,1.085900
60,0.811300
80,0.708500
100,0.610300
120,0.703000
140,0.596900
160,0.620100
180,0.636300
200,0.594700


TrainOutput(global_step=2163, training_loss=0.36555433642572127, metrics={'train_runtime': 1183.5651, 'train_samples_per_second': 29.223, 'train_steps_per_second': 1.828, 'total_flos': 2275075944739584.0, 'train_loss': 0.36555433642572127, 'epoch': 3.0})

# Example after training

In [20]:
print(model.config.id2label)


{0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# label mappings:
labels_a = {0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}

def classify_comment(text, labels=labels_a):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    print("Probabilities:", {labels[i]: float(probs[i]) for i in range(len(probs))})

    pred_idx = probs.argmax()
    pred_label = labels[pred_idx]
    pred_score = probs[pred_idx]

    return pred_label, pred_score

def final_verdict_sentiment(text):
    label, score = classify_comment(text, labels=labels_a)

    verdict = "TOXIC" if label == "NEGATIVE" else "NON-TOXIC"
    return {"label": label, "score": score}, verdict


result, verdict = final_verdict_sentiment("Ты лучший!")
print(result)
print(verdict)


Probabilities: {'NEUTRAL': 0.9021552801132202, 'POSITIVE': 0.09686737507581711, 'NEGATIVE': 0.0009773726342245936}
{'label': 'NEUTRAL', 'score': np.float32(0.9021553)}
NON-TOXIC
